In [3]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup



In [4]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
!pip install tensorflow_datasets

  Created wheel for promise: filename=promise-2.3-cp37-none-any.whl size=21500 sha256=2610cf59a35712069652463359571cf3790a8e6fa19adb6a07870c3b8063a3bb
  Stored in directory: C:\Users\UA\AppData\Local\pip\Cache\wheels\19\49\34\c3c1e78bcb954c49e5ec0d31784fe63d14d427f316b12fbde9
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78598 sha256=248f64d308ae04491fccec20f4bde915cd137ec0cb3587bb0e03877568bfed9a
  Stored in directory: C:\Users\UA\AppData\Local\pip\Cache\wheels\59\b1\91\f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-cp37-none-any.whl size=77603 sha256=5eae5b704d2a0b476be95437ab7c9d26492466240c5f6ed9d3c51b9ec7ecde43
  Stored in directory: C:\Users\UA\AppData\Local\pip\Cache\wheels\2c\f9\7f\6eb87e636072bf467e25348bbeb96849333e6a080dca78f706
Successfully built promise dill googleapis-common-protos


In [5]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
train_data = pd.read_csv(
    r"C:\Users\UA\train.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)
test_data = pd.read_csv(
    r"C:\Users\UA\test.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [6]:
data = train_data

In [7]:
data.drop(["id", "date", "query", "user"], # don't forget to run data = train_data before!
          axis=1,
          inplace=True)

In [8]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

C:\Users\UA\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'Cookies '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\UA\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b' i just received my G8 viola exam.. and its... well... .. disappointing.. :\\..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\UA\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'PICTURES '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\UA\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \\../  \\../'" looks like a filename, not markup. You should probably open this file and pass the f

In [10]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [11]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [12]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [13]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [14]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

In [15]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() # no training variable so we can
                                             # use the same layer for each
                                             # pooling step
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [16]:


VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2#len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [17]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [18]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [19]:
checkpoint_path = r"C:\Users\UA\train.csv\ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [20]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Train on 1584080 samples
Epoch 1/5


   5408/1584080 [..............................] - ETA: 84:37:02 - loss: 0.6939 - accuracy: 0.56 - ETA: 44:09:22 - loss: 0.7005 - accuracy: 0.48 - ETA: 30:21:48 - loss: 0.6946 - accuracy: 0.51 - ETA: 23:25:57 - loss: 0.6937 - accuracy: 0.50 - ETA: 19:16:00 - loss: 0.6939 - accuracy: 0.49 - ETA: 16:29:05 - loss: 0.6921 - accuracy: 0.52 - ETA: 14:28:59 - loss: 0.6911 - accuracy: 0.53 - ETA: 13:01:08 - loss: 0.6909 - accuracy: 0.54 - ETA: 11:51:25 - loss: 0.6901 - accuracy: 0.56 - ETA: 10:56:24 - loss: 0.6894 - accuracy: 0.56 - ETA: 10:10:51 - loss: 0.6899 - accuracy: 0.55 - ETA: 9:32:00 - loss: 0.6928 - accuracy: 0.5339 - ETA: 8:59:38 - loss: 0.6931 - accuracy: 0.533 - ETA: 8:32:33 - loss: 0.6930 - accuracy: 0.533 - ETA: 8:08:24 - loss: 0.6926 - accuracy: 0.533 - ETA: 7:47:27 - loss: 0.6920 - accuracy: 0.539 - ETA: 7:28:48 - loss: 0.6920 - accuracy: 0.534 - ETA: 7:12:11 - loss: 0.6911 - accuracy: 0.539 - ETA: 6:57:33 - loss: 0.6905 - accuracy: 0.537 - ETA: 6:44:06 - loss: 0.6900 - accura

  10848/1584080 [..............................] - ETA: 3:03:12 - loss: 0.6176 - accuracy: 0.647 - ETA: 3:03:00 - loss: 0.6165 - accuracy: 0.647 - ETA: 3:02:47 - loss: 0.6161 - accuracy: 0.648 - ETA: 3:02:34 - loss: 0.6153 - accuracy: 0.648 - ETA: 3:02:24 - loss: 0.6149 - accuracy: 0.649 - ETA: 3:02:15 - loss: 0.6138 - accuracy: 0.650 - ETA: 3:02:05 - loss: 0.6135 - accuracy: 0.650 - ETA: 3:01:53 - loss: 0.6135 - accuracy: 0.651 - ETA: 3:01:45 - loss: 0.6135 - accuracy: 0.651 - ETA: 3:01:35 - loss: 0.6131 - accuracy: 0.651 - ETA: 3:01:26 - loss: 0.6122 - accuracy: 0.653 - ETA: 3:01:17 - loss: 0.6127 - accuracy: 0.652 - ETA: 3:01:06 - loss: 0.6123 - accuracy: 0.652 - ETA: 3:00:56 - loss: 0.6120 - accuracy: 0.653 - ETA: 3:00:44 - loss: 0.6121 - accuracy: 0.653 - ETA: 3:00:34 - loss: 0.6118 - accuracy: 0.653 - ETA: 3:00:27 - loss: 0.6115 - accuracy: 0.654 - ETA: 3:00:16 - loss: 0.6115 - accuracy: 0.654 - ETA: 3:00:05 - loss: 0.6114 - accuracy: 0.654 - ETA: 2:59:56 - loss: 0.6112 - accurac

  16288/1584080 [..............................] - ETA: 2:46:47 - loss: 0.5771 - accuracy: 0.689 - ETA: 2:46:46 - loss: 0.5766 - accuracy: 0.690 - ETA: 2:46:43 - loss: 0.5765 - accuracy: 0.690 - ETA: 2:46:40 - loss: 0.5766 - accuracy: 0.690 - ETA: 2:46:38 - loss: 0.5763 - accuracy: 0.690 - ETA: 2:46:35 - loss: 0.5760 - accuracy: 0.690 - ETA: 2:46:30 - loss: 0.5758 - accuracy: 0.690 - ETA: 2:46:27 - loss: 0.5753 - accuracy: 0.690 - ETA: 2:46:25 - loss: 0.5749 - accuracy: 0.691 - ETA: 2:46:22 - loss: 0.5750 - accuracy: 0.691 - ETA: 2:46:19 - loss: 0.5749 - accuracy: 0.691 - ETA: 2:46:18 - loss: 0.5748 - accuracy: 0.691 - ETA: 2:46:14 - loss: 0.5748 - accuracy: 0.691 - ETA: 2:46:12 - loss: 0.5746 - accuracy: 0.691 - ETA: 2:46:09 - loss: 0.5747 - accuracy: 0.691 - ETA: 2:46:04 - loss: 0.5745 - accuracy: 0.691 - ETA: 2:46:01 - loss: 0.5743 - accuracy: 0.692 - ETA: 2:45:59 - loss: 0.5746 - accuracy: 0.692 - ETA: 2:45:58 - loss: 0.5742 - accuracy: 0.692 - ETA: 2:45:55 - loss: 0.5739 - accurac

  21728/1584080 [..............................] - ETA: 2:41:01 - loss: 0.5595 - accuracy: 0.705 - ETA: 2:41:00 - loss: 0.5595 - accuracy: 0.705 - ETA: 2:41:00 - loss: 0.5593 - accuracy: 0.705 - ETA: 2:40:59 - loss: 0.5594 - accuracy: 0.705 - ETA: 2:40:58 - loss: 0.5593 - accuracy: 0.705 - ETA: 2:40:56 - loss: 0.5590 - accuracy: 0.705 - ETA: 2:40:55 - loss: 0.5587 - accuracy: 0.705 - ETA: 2:40:52 - loss: 0.5589 - accuracy: 0.705 - ETA: 2:40:51 - loss: 0.5585 - accuracy: 0.705 - ETA: 2:40:50 - loss: 0.5584 - accuracy: 0.706 - ETA: 2:40:48 - loss: 0.5580 - accuracy: 0.706 - ETA: 2:40:47 - loss: 0.5576 - accuracy: 0.706 - ETA: 2:40:47 - loss: 0.5577 - accuracy: 0.706 - ETA: 2:40:45 - loss: 0.5574 - accuracy: 0.706 - ETA: 2:40:44 - loss: 0.5573 - accuracy: 0.706 - ETA: 2:40:43 - loss: 0.5574 - accuracy: 0.706 - ETA: 2:40:42 - loss: 0.5573 - accuracy: 0.706 - ETA: 2:40:41 - loss: 0.5572 - accuracy: 0.706 - ETA: 2:40:40 - loss: 0.5569 - accuracy: 0.706 - ETA: 2:40:39 - loss: 0.5567 - accurac

...

 554848/1584080 [=========>....................] - ETA: 1:44:55 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:55 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:55 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:54 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:54 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:54 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:54 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:53 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:53 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:53 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:53 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:52 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:52 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:52 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:52 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:51 - loss: 0.4245 - accuracy: 0.804 - ETA: 1:44:51 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:51 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:51 - loss: 0.4244 - accuracy: 0.804 - ETA: 1:44:51 - loss: 0.4244 - accurac

 560288/1584080 [=========>....................] - ETA: 1:44:14 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:14 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:13 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:13 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:13 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:13 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:12 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:12 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:12 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:12 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:11 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:11 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:11 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:11 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:10 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:10 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:10 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:10 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:09 - loss: 0.4241 - accuracy: 0.804 - ETA: 1:44:09 - loss: 0.4241 - accurac

 565728/1584080 [=========>....................] - ETA: 1:43:34 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:34 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:34 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:34 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:33 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:33 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:33 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:33 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:32 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:32 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:32 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:32 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:32 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:31 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:31 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:31 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:31 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:30 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:30 - loss: 0.4238 - accuracy: 0.804 - ETA: 1:43:30 - loss: 0.4238 - accurac

 569792/1584080 [=========>....................] - ETA: 1:42:56 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:56 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:55 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:55 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:55 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:55 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:54 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:54 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:54 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:54 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:53 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:53 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:53 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:53 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:52 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:52 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:52 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:52 - loss: 0.4235 - accuracy: 0.804 - ETA: 1:42:52 - loss: 0.4236 - accuracy: 0.804 - ETA: 1:42:51 - loss: 0.4236 - accurac

KeyboardInterrupt: 

In [21]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

16000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

 - 12s 724us/sample - loss: 0.4141 - accuracy: 0.8227
[0.391905391395092, 0.8226875]


In [25]:
Dcnn(np.array([tokenizer.encode("bad teacher")]), training=False, batch_size=1, epochs=1).numpy()

TypeError: call() got an unexpected keyword argument 'batch_size'

In [23]:
tokenizer.encode("bad")

[621]